In [2]:
import hashlib
import string
import json
import random
import logging
import numpy as np
import pandas as pd
import datetime
import binascii
import pylab as pl
import base64
import collections
import Crypto
from Crypto import Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [3]:
transactions = []

In [4]:
class Client:
    def __init__(self):
        random = Random.new().read
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        
    @property
    def identity(self):
        """NOTE: The key provided in hexlify func. should be byte like"""
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
    

In [5]:
Dinesh = Client()
print(Dinesh.identity)

30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd1f848399ece41b5cbf47e4bd1fbc2c4d551080dc14296a427206d24cad113bebae98c5b51df50203010001


In [6]:
class Transaction:
    def __init__(self, sender, recipient, value):
        """
        sender: Sender Class Object.
        recipient: 
        value: It is the amount of money send.
        """
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
    
    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity
        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time': self.time
        })
    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        digest_hash = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(digest_hash)).decode('ascii')
    

In [7]:
Dinesh.identity

'30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd1f848399ece41b5cbf47e4bd1fbc2c4d551080dc14296a427206d24cad113bebae98c5b51df50203010001'

In [8]:
Naresh = Client()
Naresh.identity

'30819f300d06092a864886f70d010101050003818d0030818902818100af4ccf2de98b4a1795cac18978bef976ab4dfdd967f6eb16aa65575d3f156681dfffd296a21092e01347ce98d8c960c19803f64606bc43997575de71c2d10dbcaed7d0ce242c14488aa90862198bcf9474ea351632e726e041a945c67041df3af3503c729460bf4ca21a227d1c49ce2dade11638bc170dfe4b8ebc7b1fe925250203010001'

In [9]:
t = Transaction(
    Dinesh,
    Naresh.identity,
    10.25
)

In [10]:
signer = t.sign_transaction()
print(signer)

4152a8dd9a75cc327a6759ee4d09f0155f583b39bbfef6c8f0dc05a0cbd102019cbb1362a780a8847b7afcaad39e7e9ab747045090b9bef23220413f37d810556c2b1ff4a5e8701add9373dd43f682d568764f244d459b119e7eab4312737848b00d5af67747405f3f266134eb405bc21a5e4c8be84ed2779c9f240fc28e90a3


In [11]:
Seema = Client()
Manoj = Client()
print("Dinesh: \n", Dinesh.identity, "Naresh: \n", Naresh.identity\
     , "Seema: \n", Seema.identity, "Manoj: \n", Manoj.identity)


Dinesh: 
 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd1f848399ece41b5cbf47e4bd1fbc2c4d551080dc14296a427206d24cad113bebae98c5b51df50203010001 Naresh: 
 30819f300d06092a864886f70d010101050003818d0030818902818100af4ccf2de98b4a1795cac18978bef976ab4dfdd967f6eb16aa65575d3f156681dfffd296a21092e01347ce98d8c960c19803f64606bc43997575de71c2d10dbcaed7d0ce242c14488aa90862198bcf9474ea351632e726e041a945c67041df3af3503c729460bf4ca21a227d1c49ce2dade11638bc170dfe4b8ebc7b1fe925250203010001 Seema: 
 30819f300d06092a864886f70d010101050003818d0030818902818100c16f6d8a425945b5416fd2f8ed4bda1cdc117a6f67b34b8fba8c95b06a6ea4b2e1258729bd3f0619d930af7d4df8f65e0c2f32734816cde927cb2bb9c677aa5590cc47aeb48e9f0e92fe3b19bd516a8010e9ceb3616f011dd38ce0c3bc55cf504e8247afc00e5408ef9e64dd7637f4a78cf2637be83746795c3b77aa482511490203010

In [12]:
t1 = Transaction(Dinesh, Naresh.identity, 15.25)

In [13]:
signature_1 = t1.sign_transaction()
print("From Dinesh to Naresh: \n", signature_1)

From Dinesh to Naresh: 
 4c0e88031e9655026712bf254d83dce48d753320133cf57611786765011ec5b97203c8e5d8477e0eb7ecf1852fbef69fba0c92af4d3eae5c0b417d5782a7e284103c5b93afbccbe4aa8df287c61072f52286e18a78df7f1e6d3d8e2a3080c6a7830c262453787b6279888fe59738f8233827236edda46861b771c12646eb0e71


In [14]:
transactions.append(t1)

In [15]:
t2 = Transaction(
   Dinesh,
   Seema.identity,
   6.0
)

In [16]:
t2.sign_transaction()
transactions.append(t2)

In [17]:
def display_transaction(transaction):
   #for transaction in transactions:
   dict = transaction.to_dict()
   print ("sender: " + dict['sender'])
   print ('-----')
   print ("recipient: " + dict['recipient'])
   print ('-----')
   print ("value: " + str(dict['value']))
   print ('-----')
   print ("time: " + str(dict['time']))
   print ('-----')

In [18]:
for transaction in transactions:
   display_transaction (transaction)
   print ('--------------')

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd1f848399ece41b5cbf47e4bd1fbc2c4d551080dc14296a427206d24cad113bebae98c5b51df50203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100af4ccf2de98b4a1795cac18978bef976ab4dfdd967f6eb16aa65575d3f156681dfffd296a21092e01347ce98d8c960c19803f64606bc43997575de71c2d10dbcaed7d0ce242c14488aa90862198bcf9474ea351632e726e041a945c67041df3af3503c729460bf4ca21a227d1c49ce2dade11638bc170dfe4b8ebc7b1fe925250203010001
-----
value: 15.25
-----
time: 2020-05-11 03:26:16.395303
-----
--------------
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd

In [19]:
BLOCK_LENGTH = 3
last_block_hash = ""

In [20]:
class Block:
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = ""

In [21]:
"""
Seema, Manoj, Dinesh, Naresh"""
t3 = Transaction(Manoj, Naresh.identity, 12.95)
t3.sign_transaction()
transactions.append(t3)
t4 = Transaction(Manoj, Seema.identity, 129)
t4.sign_transaction()
transactions.append(t4)
t5 = Transaction(Seema, Naresh.identity, 15.0)
t5.sign_transaction()
transactions.append(t5)
t6 = Transaction(Manoj, Seema.identity, 100)
t6.sign_transaction()
transactions.append(t6)
t7 = Transaction(Manoj, Naresh.identity, 5)
t7.sign_transaction()
transactions.append(t7)


In [22]:
block0 = Block()

In [23]:
block0.previous_block_hash = None
block0.Nonce = None
# verified_transaction_list is the list of transaction, a
# Block Contains. {MAX/MIN=3}

In [24]:
block0.verified_transactions.append(t1)
block0.verified_transactions.append(t2)
block0.verified_transactions.append(t3)
display_transaction(t1)

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd1f848399ece41b5cbf47e4bd1fbc2c4d551080dc14296a427206d24cad113bebae98c5b51df50203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100af4ccf2de98b4a1795cac18978bef976ab4dfdd967f6eb16aa65575d3f156681dfffd296a21092e01347ce98d8c960c19803f64606bc43997575de71c2d10dbcaed7d0ce242c14488aa90862198bcf9474ea351632e726e041a945c67041df3af3503c729460bf4ca21a227d1c49ce2dade11638bc170dfe4b8ebc7b1fe925250203010001
-----
value: 15.25
-----
time: 2020-05-11 03:26:16.395303
-----


In [25]:
Sikka = []

In [26]:
Sikka.append(block0)

In [27]:
last_block_hash = hashlib.sha256(str(block0).encode('ascii')).hexdigest()
print(last_block_hash)

57b473dad3979b344f0924808e67c132f1d488c53e04d75d8f4dc73bd6498ef1


In [28]:
def dump_chain(chain):
    print("No. of blocks in Sikka: ", len(chain))
    for i in range(len(chain)):
        temp_block = chain[i]
        print("Block # " + str(i))
        for t in temp_block.verified_transactions:
            display_transaction(t)
            print("------------END------------")
        print("===========END OF BLOCK============")

In [29]:
dump_chain(Sikka)

No. of blocks in Sikka:  1
Block # 0
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd1f848399ece41b5cbf47e4bd1fbc2c4d551080dc14296a427206d24cad113bebae98c5b51df50203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100af4ccf2de98b4a1795cac18978bef976ab4dfdd967f6eb16aa65575d3f156681dfffd296a21092e01347ce98d8c960c19803f64606bc43997575de71c2d10dbcaed7d0ce242c14488aa90862198bcf9474ea351632e726e041a945c67041df3af3503c729460bf4ca21a227d1c49ce2dade11638bc170dfe4b8ebc7b1fe925250203010001
-----
value: 15.25
-----
time: 2020-05-11 03:26:16.395303
-----
------------END------------
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc

In [31]:
Block1 = Block()
Block1.previous_block_hash = last_block_hash
Block1.verified_transactions.append(t4)
Block1.verified_transactions.append(t5)
Block1.verified_transactions.append(t6)

In [39]:
for i in range(len(Block1.verified_transactions)):
    print("Transaction: #", i + 1)
    display_transaction(Block1.verified_transactions[i])

Transaction: # 1
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100b08b8a52e0401b421cdc09ad9da3fddc13bc823b8b36df3d04bc7ce40b5d063a28c0084a6d803f680f97f8928db9a7b6b08a3ef98af403b389b4f03ef861b252b3d012e6512697772758fe87cfe7f1d2b34ea27e1379871c65cd0b46e34a26ebb06a248913a5583de74e35b438aac923c61185619732d5bcaab5df45920333a10203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c16f6d8a425945b5416fd2f8ed4bda1cdc117a6f67b34b8fba8c95b06a6ea4b2e1258729bd3f0619d930af7d4df8f65e0c2f32734816cde927cb2bb9c677aa5590cc47aeb48e9f0e92fe3b19bd516a8010e9ceb3616f011dd38ce0c3bc55cf504e8247afc00e5408ef9e64dd7637f4a78cf2637be83746795c3b77aa482511490203010001
-----
value: 129
-----
time: 2020-05-11 03:26:18.684744
-----
Transaction: # 2
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100c16f6d8a425945b5416fd2f8ed4bda1cdc117a6f67b34b8fba8c95b06a6ea4b2e1258729bd3f0619d930af7d4df8f65e0c2f32734816cde927cb2bb9c677aa5590cc47aeb48e9f0e92fe3b19bd516a8010e

In [37]:
Block1.verified_transactions.pop()

0

In [40]:
Sikka.append(Block1)

In [41]:
dump_chain(Sikka)

No. of blocks in Sikka:  2
Block # 0
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc82b56dc090df3c1e18e220177640ab47395318f2d866b790bd1f848399ece41b5cbf47e4bd1fbc2c4d551080dc14296a427206d24cad113bebae98c5b51df50203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100af4ccf2de98b4a1795cac18978bef976ab4dfdd967f6eb16aa65575d3f156681dfffd296a21092e01347ce98d8c960c19803f64606bc43997575de71c2d10dbcaed7d0ce242c14488aa90862198bcf9474ea351632e726e041a945c67041df3af3503c729460bf4ca21a227d1c49ce2dade11638bc170dfe4b8ebc7b1fe925250203010001
-----
value: 15.25
-----
time: 2020-05-11 03:26:16.395303
-----
------------END------------
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a5453ea002bb69527c691a72043a3647bb7d0c841fbbb128bdbd98e64bb91f281484c173d11eb0c39298ebcd91b109e166937cdda980ba6af39803cf198b2f07cc